# Sentinel — Validation, Sweep Ranking & Trade Journal

This notebook connects Sentinel end-to-end:

**Goals**
1. Sanity-check model predictions on a held-out sample of sweeps  
2. Rank historical sweeps by a simple “edge score” using direction + expected move  
3. Export a lightweight **Excel trade journal template** for review

**Important**
This is **not** a live trading system. It demonstrates:
- Correct model wiring (load → features → predict)
- Practical interpretation (ranking)
- A simple analyst workflow (journal export)

## 1. Setup: imports and project paths

This cell:
- Imports the core libraries used for evaluation + ranking
- Defines **consistent paths** to the project root, model artifacts, the parquet dataset, and an outputs folder

If this notebook runs, it proves your repo structure is coherent and reproducible.

In [1]:
from pathlib import Path
import joblib
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, mean_absolute_error

# Paths
PROJECT_ROOT = Path("..").resolve()
MODELS_DIR = PROJECT_ROOT / "models"
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "tradyflow_training.parquet"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"

OUTPUTS_DIR.mkdir(exist_ok=True)

## 2. Load trained models + the engineered sweep dataset

This cell:
- Loads the three trained Sentinel models from `models/`
- Loads the engineered dataset from `data/processed/tradyflow_training.parquet`

If these load successfully, it confirms:
- Artifacts exist
- Paths are correct
- Your notebook environment matches the project assumptions

In [2]:
# Load models
direction_rf = joblib.load(MODELS_DIR / "sentinel_direction_up_rf.pkl")
volregime_rf = joblib.load(MODELS_DIR / "sentinel_vol_regime_rf.pkl")
nextret_rf = joblib.load(MODELS_DIR / "sentinel_next_return_rf.pkl")

print("Models loaded")

# Load data
df = pd.read_parquet(DATA_PATH)
print(f"Loaded {len(df)} rows")

Models loaded
Loaded 6704 rows


## 3. Recreate the feature set used in training

The models expect the **same numeric feature columns** used during training.

This helper:
- Excludes ID / metadata columns and the target labels
- Selects numeric columns only
- Returns the feature list used to build `X`

This is the “don’t silently break inference” guardrail.

In [3]:
def get_feature_cols(df):
    exclude_cols = {
        "Time",
        "Sym",
        "C/P",
        "Exp",
        "next_spot",
        "next_return_1d",
        "direction_up",
        "vol_regime",
    }
    numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns
    return [c for c in numeric_cols if c not in exclude_cols]

feature_cols = get_feature_cols(df)
len(feature_cols), feature_cols[:10]

(18,
 ['Strike',
  'Spot',
  'BidAsk',
  'Orders',
  'Vol',
  'Prems',
  'OI',
  'Diff(%)',
  'ITM',
  'moneyness'])

## 4. Validation: quick sanity metrics (not a full backtest)

Here we compute quick “does this wiring make sense?” checks:
- Direction model: accuracy vs. `direction_up`
- Vol regime model: accuracy vs. `vol_regime`
- Return model: MAE vs. `next_return_1d`

This is not meant to be a publication-grade evaluation—just a fast correctness check.

In [4]:
X = df[feature_cols]

# Direction
y_dir = df["direction_up"]
dir_pred = direction_rf.predict(X)
dir_acc = accuracy_score(y_dir, dir_pred)

# Vol regime
y_vol = df["vol_regime"]
vol_pred = volregime_rf.predict(X)
vol_acc = accuracy_score(y_vol, vol_pred)

# Return
y_ret = df["next_return_1d"]
ret_pred = nextret_rf.predict(X)
ret_mae = mean_absolute_error(y_ret, ret_pred)

dir_acc, vol_acc, ret_mae

(0.9201968973747017, 1.0, 0.04770199003197451)

## 5. Ranking historical sweeps (a practical analyst view)

This cell builds a simple ranking table to answer:

> “Which sweeps *historically* looked most interesting?”

We:
- Predict `p_up` (directional bias)
- Predict `predicted_return` (expected next-day move)
- Combine them into a simple **edge_score** that rewards:
  - Strong directional tilt (away from 50/50)
  - Larger expected move magnitude

Result: a ranked list you can skim like a “watchlist,” not a trade signal.

In [5]:
rank_df = df.copy()

rank_df["p_up"] = direction_rf.predict_proba(X)[:, 1]
rank_df["predicted_return"] = ret_pred
rank_df["abs_return"] = rank_df["predicted_return"].abs()

# Simple composite score
rank_df["edge_score"] = (
    rank_df["p_up"] * 0.6 +
    rank_df["abs_return"].rank(pct=True) * 0.4
)

rank_df_sorted = rank_df.sort_values("edge_score", ascending=False)

rank_df_sorted[
    ["Sym", "C/P", "Strike", "p_up", "predicted_return", "edge_score"]
].head(10)

,Sym,C/P,Strike,p_up,predicted_return,edge_score
5398,SPIR,Call,12.5,0.890000,0.235597,0.926422
2002,DT,Put,60.0,0.953302,0.111108,0.926217
3019,IONQ,Call,25.0,0.923563,0.138243,0.924365
2515,FUBO,Call,31.0,0.900103,0.183815,0.924310
554,ATER,Call,10.0,0.877214,0.270001,0.921913
4804,QCOM,Put,120.0,0.904714,0.158917,0.921468
4507,PDD,Call,95.0,0.884770,0.205910,0.919346
1373,CMPS,Call,40.0,0.869628,0.272784,0.917779
4485,PBR,Put,10.0,0.896192,0.158935,0.916414
3530,LYV,Put,95.0,0.918321,0.124626,0.914358


### How to read this ranking

- **p_up**: model’s probability the next move is up
- **predicted_return**: expected size/direction of next-day move
- **edge_score**: blended ranking metric (direction strength × move magnitude)

This is a **ranking tool**, not a trade list. It helps prioritize review.

## 6. Create a lightweight trade journal template (Excel)

This section creates a simple journal structure you can use outside the app.

We:
- Define the journal columns (date, symbol, thesis, entry/exit, P&L, notes, etc.)
- Populate one example row from the top-ranked sweep
- Export to an `.xlsx` file for quick review/editing

This is intentionally minimal: credibility > complexity.

In [6]:
journal_cols = [
    "date",
    "symbol",
    "call_put",
    "strike",
    "directional_bias",
    "expected_move",
    "thesis",
    "entry",
    "exit",
    "pnl",
    "notes",
]

journal_df = pd.DataFrame(columns=journal_cols)
journal_df

,date,symbol,call_put,strike,directional_bias,expected_move,thesis,entry,exit,pnl,notes


## 7. Export the journal to Excel

This writes the journal to `outputs/sentinel_trade_journal.xlsx`.

Note: Pandas requires an Excel engine (typically `openpyxl`) to write `.xlsx` files.
If you see `ModuleNotFoundError: openpyxl`, install it or add it to requirements.

In [7]:
top = rank_df_sorted.iloc[0]

example_row = {
    "date": pd.Timestamp.today().date(),
    "symbol": top.get("Sym"),
    "call_put": top.get("C/P"),
    "strike": top.get("Strike"),
    "directional_bias": round(top["p_up"], 3),
    "expected_move": round(top["predicted_return"], 4),
    "thesis": "High flow + favorable direction",
    "entry": "",
    "exit": "",
    "pnl": "",
    "notes": "",
}

journal_df = pd.concat(
    [journal_df, pd.DataFrame([example_row])],
    ignore_index=True
)

journal_df

/tmp/ipykernel_2607/3074328038.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  journal_df = pd.concat(


,date,symbol,call_put,strike,directional_bias,expected_move,thesis,entry,exit,pnl,notes
0,2025-12-16,SPIR,Call,12.5,0.89,0.2356,High flow + favorable direction,,,,


In [9]:
output_path = OUTPUTS_DIR / "sentinel_trade_journal.xlsx"
journal_df.to_excel(output_path, index=False)

output_path

PosixPath('/home/btheard/projects/sentinel/outputs/sentinel_trade_journal.xlsx')

## What this notebook proves

- Models load correctly from `models/`
- Feature selection is consistent with training expectations
- Predictions run end-to-end
- Sweeps can be ranked into a practical review list
- A journal workflow exists (Excel export)

This is the “portfolio completeness” glue for Sentinel Baseline.